# Context
This notebook drives the training process for different models.

In [ ]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../project.env")
sys.path.append(os.environ["PYTHONPATH"])

import pickle

In [ ]:
# Import project-wide and PH2 specific variables and functions
import superheader as sup
import TRAINheader as train

# Datasets

In [ ]:
import pandas as pd

In [ ]:
sup.report_dir_if_not_exists(sup.PH3_DATA_ROOT)

# Model training

## KNN

### Active Hand prediction

In [ ]:
train.arch.knn.best_KNN(sup.DATA_AH_PF, sup.active_hand_col, "two-classes", 0.2)

### Sign prediction

In [ ]:
train.arch.knn.best_KNN(sup.DATA_S_PF, sup.class_numeric_column, "two-classes", 0.2)

In [ ]:
train.arch.knn.best_KNN(sup.DATA_S_PV, sup.class_numeric_column, "two-classes", 0.2)

# SCORE ANALYSIS

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
knn_scores_df = pd.DataFrame(sup.knn_score_tracker, columns=sup.knn_scores_columns)

In [ ]:
knn_scores_df

In [ ]:
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, sup.TRAIN_KNN_CODE))
knn_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT, sup.TRAIN_KNN_CODE, f"{now}.csv"), index=False)

## Active hand detection

In [ ]:
ah_scores = knn_scores_df[knn_scores_df['data_unit'] == sup.DATA_AH_PF]

## Sign recognition

### Only per frame data

In [ ]:
s_pf_scores = knn_scores_df[knn_scores_df['data_unit'] == sup.DATA_S_PF]

### Only per video data

In [ ]:
s_pv_scores = knn_scores_df[knn_scores_df['data_unit'] == sup.DATA_S_PV]